<a href="https://colab.research.google.com/github/jadepanths/Python-PyTorch-Tutorial/blob/main/Build_The_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build The Neural Network
The Neural networks comprise of layer or modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network. All the modules in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is modile itself that consists of other modules/laters. This nested structure allows for building and managing complex architectures easily.<br/>
<br/>
We will build a neural network to classify images in the FasionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for training
Training a model on a hardware accelerator like a GPU is faster than traing on a cpu. Therefore, we should check if cuda is available.

__Note:__ To enable cuda, go to "Runtime" -> "change runtime type" -> change hardware accelerator to "GPU".

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


# Define the Class
We define our neural network bu subclassing _nn.Module_, and initialize the neural network laters in ```__init__```. Every _nn.Module_ subclass implements the operations on input data in the _forward_ method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

After creating an instance if _NeuralNetwork_, we can move it to the _device_ and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Note: **ReLu** is like Riemann sums. You can approximate any continuos functions with lots of little reactangles. ReLu activations can produced lots of little rectangles. ReLu can make complicated shapes and approximate maby complicated domains.

To use the model, we pass it the input data. This executes the model’s _forward_, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call model.forward() directly. <br/>
<br/>
Calling the model on the input returns a 10-dimensional tensor with a raw predicted values for each class. We get the prediction probabilities by passign ti through an instance of the ```nn.Softmax``` module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


# Model Layers
Break down the layers in the FashionMNIST model. We will take a sample minibatch of 3 images of size 28x28.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28*28 image into a contiguous of 784 pixel values (the minibatch dimension (at dim=0) is maintained)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## Linear
The [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## ReLU
[nn.ReLu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) is used between the linear layers. (There is other activations to introduce non-linearity in your model.

Non-linear activations create the complex mapping between the model's input and outputs. They are apllied after the linear transformation to introduce _nonlinearity_.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1995, -0.7510,  0.2033, -0.3022,  0.1698, -0.3047, -0.1637, -0.1876,
         -0.0198, -0.0042,  0.8384, -0.2327,  0.0033,  0.2341, -0.3179, -0.2469,
          0.0570,  0.0995,  0.0581, -0.4231],
        [ 0.0751, -0.9518,  0.2376, -0.3440,  0.0947,  0.0510,  0.1051, -0.4849,
          0.0120,  0.3127,  0.9142, -0.3329, -0.5684,  0.0671, -0.2668, -0.2917,
          0.2045,  0.0606,  0.1925, -0.6722],
        [-0.0261, -0.6533,  0.3344, -0.3380, -0.1327, -0.4048, -0.1737, -0.5577,
         -0.1190, -0.1278,  0.7578, -0.0967, -0.1929,  0.0274, -0.3068, -0.3184,
          0.3542,  0.0470,  0.4057, -0.6951]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.2033, 0.0000, 0.1698, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.8384, 0.0000, 0.0033, 0.2341, 0.0000, 0.0000, 0.0570, 0.0995,
         0.0581, 0.0000],
        [0.0751, 0.0000, 0.2376, 0.0000, 0.0947, 0.0510, 0.1051, 0.0000, 0.0120,
         0.3127, 0.9142, 0.0000, 0.0000, 0.0671, 0.000

## Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. This data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quicl network like *seq_modules*

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)